<a href="https://colab.research.google.com/github/AbhinavRobinson/CSE_LAB_ASSIGNMENTS/blob/master/Stock_Prediction_using_RNN_with_CUDNN_LSTM_Cells(UPDATED).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Prediction Using RNN and CuDNNLSTMs
##For Education Use Only
> Disclaimer : USE THIS FOR BETTING AT YOUR OWN RISK 

Hi there!

In this Assignment of EMAT203L , We make a stock prediction algorithm.

One way to achive this is using RNN, implemented via TensorFlow. 
We will be using TensorFlow 1.15 for this. This program may not be compatible with TensorFlow 2.X.X.
Read the Documentation carefully!

-Abhinav Robinson :)

#Method

We are only concerned with the "Close" data at this moment.
Hence we will use pandas to cleanup the data, enter data inside a Dataframe, with only the relavant columns.

In [0]:
import pandas as pd

# Lets check the data :)
main_df = pd.DataFrame()

# Lets combine all data in one dataframe
ratios = ["BJB-INR","BBA-INR","PNB-INR","YES-INR"]
for ratio in ratios:
  dataset = f"{ratio}.csv"
  # #IF new dataset obtained from LMS
  df = pd.read_csv(dataset,  
                   names=["time","open","high","low","close"],
                   encoding = "UTF-8", 
                   header = 1)
  # #ELSE if row[1] removed
#   df = pd.read_csv(dataset,  
#                    names=["time","open","high","low","close"],
#                    encoding = "UTF-8")
  df.rename(columns={"close": f"{ratio}_close"}, 
            inplace=True)
  df.set_index("time",inplace=True)
  df = df[[f"{ratio}_close"]]

  if len(main_df) == 0:
    main_df = df
  else:
    main_df = main_df.join(df)

# Lets Test our data cleanup
print(main_df.head())

            BJB-INR_close  BBA-INR_close  PNB-INR_close  YES-INR_close
time                                                                  
29-08-2019        3299.30          91.80          65.05          57.40
28-08-2019        3326.35          94.50          65.65          59.50
27-08-2019        3357.75          97.30          67.15          64.50
26-08-2019        3324.25          95.05          65.75          62.90
23-08-2019        3174.35          93.50          63.60          59.25


In [0]:
import time
# Now we define some time variables
# Based on last 20 days
SEQ_LEN = 20
# We predict next 1 day
FUTURE_PERIOD_PREDICT = 1
# on the Bank Of Baroda EQ Close
RATIO_TO_PREDICT = "BBA-INR"
# Defining no. of EPOCHS, THIS is large so we can observe the trends
EPOCHS = 256
# Defining Batch Size
BATCH_SIZE = 64
# Adding Names
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

In [0]:
def classify(current, future):
  # aka, good thing, buy
  if float(future) > float(current): return 1
  # not a good thing, maybe? ;)
  else: return 0

In [0]:
# Lets add future price as a column
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)
print(main_df[[f"{RATIO_TO_PREDICT}_close", "future"]].head())

            BBA-INR_close  future
time                             
29-08-2019          91.80   94.50
28-08-2019          94.50   97.30
27-08-2019          97.30   95.05
26-08-2019          95.05   93.50
23-08-2019          93.50   90.65


In [0]:
# add output of classify to a list shown as a column
main_df['target'] = list(map(classify, 
                             main_df[f"{RATIO_TO_PREDICT}_close"],
                             main_df["future"]))

print(main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head(10))

            BBA-INR_close  future  target
time                                     
29-08-2019          91.80   94.50       1
28-08-2019          94.50   97.30       1
27-08-2019          97.30   95.05       0
26-08-2019          95.05   93.50       0
23-08-2019          93.50   90.65       0
22-08-2019          90.65   94.85       1
21-08-2019          94.85   98.70       1
20-08-2019          98.70  102.60       1
19-08-2019         102.60  102.45       0
16-08-2019         102.45  101.10       0


In [0]:
# Noramlly data is sorted, but just in case
# .values converts index values to numpy array
times = sorted(main_df.index.values)
# now we seperate the last 5% data for test data
# last_5pct is a unix timestamp
last_5pct = times[-int(0.05*len(times))]

# Now we split the data
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

In [0]:
# Import some more libraries ;_;
from sklearn import preprocessing
from collections import deque
import random
import numpy as np

# Preprocess DF to scale all values of diff. coins on one scale
# and add percent change
def preprocess_df(df):
  df = df.drop('future',1)

  for col in df.columns:
    if col != 'target':
      df[col] = df[col].pct_change()
      df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values)

  df.dropna(inplace=True)

  sequential_data = []
  prev_days = deque(maxlen = SEQ_LEN)
  print(df.head())

  for i in df.values:
    prev_days.append([n for n in i[:-1]]) 
    if len(prev_days) == SEQ_LEN:
      sequential_data.append([np.array(prev_days), i[-1]])

  random.shuffle(sequential_data)

  buys = []
  sells = []

  for seq, target, in sequential_data:
    if target == 0:
      sells.append([seq, target])
    elif target == 1:
      buys.append([seq, target])

  random.shuffle(buys)
  random.shuffle(sells)

  lower = min(len(buys), len(sells))

  # Upto 30K
  buys = buys[:lower]
  sells = sells[:lower]

  sequential_data = buys+sells
  random.shuffle(sequential_data)

  X = []
  y = []

  for seq,target in sequential_data:
    X.append(seq)
    y.append(target)

  return np.array(X), y

In [0]:
train_x , train_y = preprocess_df(main_df)
validation_x , validation_y = preprocess_df(validation_main_df)

            BJB-INR_close  BBA-INR_close  PNB-INR_close  YES-INR_close  target
time                                                                          
23-08-2019      -1.930195      -0.663428      -1.330491      -1.937743       0
22-08-2019      -0.763274      -1.230598      -1.374633      -1.663351       1
21-08-2019       2.199421       1.843056       1.664843       5.377373       1
20-08-2019       0.532283       1.613307       0.941631       2.948053       1
19-08-2019       0.545485       1.570222       1.010106       2.543769       0
            BJB-INR_close  BBA-INR_close  PNB-INR_close  YES-INR_close  target
time                                                                          
30-04-2019      -0.817659      -1.451630       0.218579       0.657223       1
31-01-2019      -1.440030      -0.412944      -0.761171       1.195323       1
30-01-2019       0.467419       0.073770      -0.126237       0.235314       1
31-12-2018       0.587590       0.422257      -0.018

In [0]:
# LETS IMPORT ALL TENSORFLOW MODULES
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, CuDNNLSTM, Dropout, BatchNormalization

#ABOUT THE MODEL

##Model
>We have formatted the model as Sequential.
###First Layers
>We are using CuDNN-LSTM which are basically CUDA optimized Long-Short Term Memory Cells, a varient of classic CPU based LSTMs.
###Additional Logic
>We use a dropout of 35% after each LSTM layer.
###Final Layers
>At the end, we boil down the model to a 32 and a 2 fully connected layer.
###Activation Functions Used
>We use activation ReLU or "Rectified Linear Unit" which is basically -> _max(0.0, x)_  
>We also use a softmax function, which is basically a _normalized exponential function_.

##Loss, Optimizer and Metrics
> We have used the sparce_categorical_crossentropy loss function to calculate loss from each epoch of training.  
> We have used RMSProp or "Root Mean Square Propagation" to perform gradient desent on this Model.   
> Our main metrics here is Accuracy.

In [0]:
DPT = 0.35
DENSE_SIZE, DENSE_DPT = 32, 0.2
model = Sequential()
model.add(CuDNNLSTM(BATCH_SIZE, 
                    input_shape=(train_x.shape[1:]), 
                    return_sequences=True))
model.add(Dropout(DPT))

model.add(CuDNNLSTM(BATCH_SIZE, 
                    input_shape=(train_x.shape[1:]), 
                    return_sequences=True))
model.add(Dropout(DPT))

model.add(CuDNNLSTM(BATCH_SIZE, 
                    input_shape=(train_x.shape[1:]), 
                    return_sequences=False))
model.add(Dropout(DPT))

model.add(Dense(DENSE_SIZE, activation='relu'))
model.add(Dropout(DENSE_DPT))

model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()


history = model.fit(
    train_x, train_y,
    batch_size= BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x,validation_y),
    verbose=1
)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_7 (CuDNNLSTM)     (None, 20, 64)            17920     
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 64)            0         
_________________________________________________________________
cu_dnnlstm_8 (CuDNNLSTM)     (None, 20, 64)            33280     
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 64)            0         
_________________________________________________________________
cu_dnnlstm_9 (CuDNNLSTM)     (None, 64)                33280     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)               

In [0]:
# AUTOMATION for training models :)
RATIO_TO_PREDICT = "PNB-INR"
validation_x , validation_y = preprocess_df(validation_main_df)
prediction = model.predict(validation_x, batch_size = 64, verbose = 0)
sum = 0
for i in prediction:
  sum += i[0]
count+=1

            BJB-INR_close  BBA-INR_close  PNB-INR_close  YES-INR_close  target
time                                                                          
30-04-2019      -0.817659      -1.451630       0.218579       0.657223       1
31-01-2019      -1.440030      -0.412944      -0.761171       1.195323       1
30-01-2019       0.467419       0.073770      -0.126237       0.235314       1
31-12-2018       0.587590       0.422257      -0.018600      -0.641941       0
30-11-2018      -0.058739      -1.236531      -0.896064      -0.470956       0


In [0]:
# We import matplotlib and create a function to plot history
import matplotlib.pyplot as plt

def plot_history(histories, key='acc'):
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])

#OBSERVATIONS

We observe that our model performs averagely on the first 400 epochs, but reaching the 512th epoch, the prediction of the validation data stays above 50%,
in other words, _we will obtain a net profit using this algorithm in the LONG game_.

Our final accuracy hovers over 65% on the data for this perticular bank.

In [0]:
# LETS NOW FIND OUT PREDICTION FOR A DIFFERENT BANK
RATIO_TO_PREDICT = "PNB-INR"
validation_x , validation_y = preprocess_df(validation_main_df)
prediction = model.predict(validation_x, batch_size = 64, verbose = 0)
for i in range(len(validation_y)):
	print("X=%s, Predicted=%s" % (validation_x[i], prediction[i]))

            BJB-INR_close  BBA-INR_close  PNB-INR_close  YES-INR_close  target
time                                                                          
30-04-2019      -0.817659      -1.451630       0.218579       0.657223       1
31-01-2019      -1.440030      -0.412944      -0.761171       1.195323       1
30-01-2019       0.467419       0.073770      -0.126237       0.235314       1
31-12-2018       0.587590       0.422257      -0.018600      -0.641941       0
30-11-2018      -0.058739      -1.236531      -0.896064      -0.470956       0


In [0]:
sum = 0
for i in prediction:
  sum += i[1]
print("The Accuracy of Prediction on a different bank is = ",sum/len(prediction)*100,"%")

The Accuracy of Prediction on a different bank is =  63.54013579504594 %


#CONCLUSION

We have generated a model which returns us with prediction accuracy of around 60% on ANY bank dataset, in other words, our transfered learning prediction is above 60%.

So you will win 6 out of 10 times on avarage, ignore local limitations.

In [0]:
#Lets Save This Model
model.save("Model_{}-{}.h5".format(sum/len(prediction)*100,count))
count+=1

In [0]:
# DEBUG VALUES
count = 0